In [ ]:
# explore the algorithm wrapped by RFE
from numpy import mean
from numpy import std

from sklearn.neighbors import KNeighborsClassifier
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold


from sklearn.feature_selection import SelectFdr, chi2, VarianceThreshold,SelectKBest,SelectPercentile
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from matplotlib import pyplot
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle

from collections import Counter

le = preprocessing.LabelEncoder()
le.fit(["CP", "NO_CP"])

In [ ]:
raw_in='../Input/raw/'
base_in='../Input/Prep/Script1/'
base_out='../Output/Script1/Results/'
glob(raw_in+'*')

In [ ]:
def div_d(my_dict,sum_p):

    #sum_p = sum(my_dict.values())

    for i in my_dict:
        my_dict[i] = float(my_dict[i]/sum_p)

    return my_dict 

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def fetch_data(exp_fname=raw_in+'TCGA-PAAD(non_norm_exp_data).csv',gene_file=base_in+'tcga_gene.csv',meta_file=raw_in+'TCGA_meta_info.csv'):
#     exp_fname=raw_in+'TCGA-PAAD(non_norm_exp_data).csv'
#     meta_file=raw_in+'TCGA_meta_info.csv'
#     gene_file=raw_in+'tcga_gene.csv'

    GSE162694_norm=pd.read_csv(exp_fname,index_col=0)
    GSE162694_norm.rename(columns={'Row.names':'Patient'},inplace=True)

    keys_index=pd.read_csv(gene_file)
 
    GSE162694_groups=pd.read_csv(meta_file)
    GSE162694_groups['Patient']=GSE162694_groups['Patient'].str.replace('-01A','')
  
    GSE162694_groups=GSE162694_groups[['Patient','Label']]
   
    maped_keys=intersection(keys_index['Genes'].to_list(),GSE162694_norm.columns.tolist())
    GSE162694_sub=GSE162694_norm[['Patient']+maped_keys].copy()

    dataset=pd.merge(GSE162694_sub,GSE162694_groups,on='Patient',   how='right')
    df=dataset.copy()
    ready_data=df.loc[:,~df.columns.duplicated()]
    #ready_data.drop('Patient',inplace=True)
    return ready_data

def feature_extract(model):
    feature_importances=[]
    importance = model.ranking_
    feature_importances = pd.Series(importance, index=model.feature_names_in_)
    sel_indx=model.support_
    my_feat=feature_importances[sel_indx]
    return my_feat,feature_importances

def feature_rank(model):
    feature_importances=[]
    importance = model.feature_importances_
    #feature_importances = pd.Series(importance, index=model.feature_names_in_)
    feature_importances = pd.DataFrame({'Name':model.feature_names_in_,'Score':importance})
    #sel_indx=model.support_
    my_feat=feature_importances.copy()
    return my_feat

In [ ]:
def get_models():
    models = dict()
    # lr
    cvs=10
    st=1
    rnd=15
    rfe = RFECV(estimator=DecisionTreeClassifier(random_state=rnd),step=st,cv=cvs,n_jobs=25)
    #model = DecisionTreeClassifier(random_state=24)
    models['dt'] = rfe
    # perceptron
    rfe = RFECV(estimator=RandomForestClassifier(random_state=2,n_estimators=50),step=st,cv=cvs,n_jobs=25)
    #model = RandomForestClassifier(random_state=24,n_estimators=50)
    models['rf'] = rfe
    # cart
    #KNN
    rfe = RFECV(estimator=Perceptron(),step=st,cv=cvs,n_jobs=25)
    #model = KNeighborsClassifier()
    models['per'] = rfe
    # rf
    rfe = RFECV(estimator=SVC(kernel='linear',random_state=rnd),step=st,cv=cvs,n_jobs=25)
    #model = SVC(kernel='linear',random_state=24)
    models['svm'] = rfe
    # gbm
    rfe = RFECV(estimator=XGBClassifier(random_state=rnd),step=st,cv=cvs,n_jobs=25)
    #model = XGBClassifier(random_state=24)
    models['xgb'] = rfe
    #Some liner
    rfe = RFECV(estimator=LogisticRegression(solver='lbfgs',max_iter=1000,random_state=rnd),step=st,cv=cvs,n_jobs=25)
    #model = LogisticRegression(solver='lbfgs',max_iter=1000,random_state=24)
    models['logit'] = rfe
    return models

# Dataset preparation

In [ ]:
#Use own dataframe having last columns as class Label
dataset=fetch_data(exp_fname=raw_in+'TCGA-PAAD(meth_data).csv',gene_file=base_in+'tcga_meth_site.csv')
#dataset.drop('Patient',inplace=True)
display(dataset.shape,dataset.head())
#######################################

# 1. Saving the FDR feature list

## Expression

In [ ]:

X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)

#X_new = SelectFdr(chi2, alpha=0.01).fit_transform(np.log2(X+1), y)
feat_names = SelectFdr(chi2, alpha=0.01).fit(np.log2(X+1), y).get_feature_names_out()
print(len(feat_names))

In [ ]:
my_features=pd.DataFrame({'Genes':list(feat_names)})
my_features.to_csv(base_in+'TCGA_FDR_feat_(a_0.1).csv',index=False)

## Mutation

# 2. saving the K best feature list

## Expression

In [ ]:

X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)

#X_new = SelectFdr(chi2, alpha=0.01).fit_transform(np.log2(X+1), y)
feat_names = SelectKBest(chi2, k=1000).fit(np.log2(X+1), y).get_feature_names_out()
print(len(feat_names))

In [ ]:
my_features=pd.DataFrame({'Genes':list(feat_names)})
my_features.to_csv(base_in+'TCGA_kbest_feat_(k_1000).csv',index=False)

# 3. saving the percentile feature list

## Expression

In [ ]:
X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)



feat_names = SelectPercentile(chi2, percentile=5).fit(np.log2(X+1), y).get_feature_names_out()
print(len(feat_names))

In [ ]:
my_features=pd.DataFrame({'Genes':list(feat_names)})
my_features.to_csv(base_in+'TCGA_p_feat_exp(p_5).csv',index=False)

## Methylation

In [ ]:

dataset=fetch_data(exp_fname=raw_in+'TCGA-PAAD(meth_data).csv',gene_file=base_in+'tcga_meth_site.csv')
#dataset.drop('Patient',inplace=True)
display(dataset.shape,dataset.head())
#######################################

In [ ]:
X=dataset.iloc[:,1:-1].dropna(axis=1)
#X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)

#X_new = SelectFdr(chi2, alpha=0.01).fit_transform(np.log2(X+1), y)
#feat_names = SelectFdr(chi2).fit(X, y).get_feature_names_out()
#selector = VarianceThreshold()
#feat_names = selector.fit(X).get_feature_names_out()
feat_names = SelectPercentile(chi2, percentile=5).fit(X, y).get_feature_names_out()
print(len(feat_names))

## Mutation

In [ ]:
dataset=fetch_data(exp_fname=raw_in+'TCGA-PAAD(mut_data).csv',gene_file=base_in+'tcga_mut_site.csv')
#dataset.drop('Patient',inplace=True)
display(dataset.shape,dataset.head())
#######################################

In [ ]:
X=dataset.iloc[:,1:-1]
#X=dataset.iloc[:,1:-1]
labels=dataset.iloc[:,-1]
y=le.transform(labels)
X.shape